# Build and test the Inference container

This notebook will provide an step-by-step instruction to create a docker image for inference module of tile-based classification and test its performance.

> Note: Before proceeding, make sure to select the correct kernel. In the top-right corner of the notebook, choose the Jupyter kernel named `Bash`.


## Setup the environment

In [3]:
export WORKSPACE=/workspace/machine-learning-process
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}
printenv | grep RUNTIME
pwd

XDG_RUNTIME_DIR=/workspace/.local
RUNTIME=/workspace/machine-learning-process/runs
/workspace/machine-learning-process/runs


## Build the container

Inspect the container file:

In [4]:
cat ${WORKSPACE}/inference/make-inference/Dockerfile

# Stage 1: Build stage
FROM rockylinux:9.3-minimal AS build

# Install necessary build tools
RUN microdnf install -y curl tar

# Download the hatch tar.gz file from GitHub
RUN curl -L https://github.com/pypa/hatch/releases/download/hatch-v1.14.0/hatch-x86_64-unknown-linux-gnu.tar.gz -o /tmp/hatch-x86_64-unknown-linux-gnu.tar.gz

# Extract the hatch binary
RUN tar -xzf /tmp/hatch-x86_64-unknown-linux-gnu.tar.gz -C /tmp/

# Stage 2: Final stage
FROM rockylinux:9.3-minimal

# Set up a default user and home directory
ENV HOME=/home/neo

# Install essential libraries including expat and python3 without `config` commands
RUN microdnf install -y \
    expat \
    libpq \
    curl \
    git \
    wget \
    tar \
    && microdnf install -y python3 \
    && microdnf clean all

# Create a user with UID 1001, group root, and a home directory
RUN useradd -u 1001 -g 100 -m -d ${HOME} -s /sbin/nologin \
         -c "Default Neo User" neo && \
    mkdir -p /code /prod ${HOME}/.cache /home/neo/.local/

Build the container using `docker`:

In [5]:
docker build --format docker -t localhost/inference:latest ${WORKSPACE}/inference/make-inference


[1/2] STEP 1/4: FROM rockylinux:9.3-minimal AS build
Resolved "rockylinux" as an alias (/etc/containers/registries.conf.d/shortnames.conf)
Trying to pull docker.io/library/rockylinux:9.3-minimal...
Getting image source signatures
Copying blob 8ec988941d66 [--------------------------------------] 0.0b / 0.0b
Copying blob 8ec988941d66 skipped: already exists  
Copying blob 8ec988941d66 skipped: already exists  
Copying config dfaa211c6b [====================================] 578.0b / 578.0b
Copying config dfaa211c6b done  
Copying config dfaa211c6b done  
Writing manifest to image destination
Storing signatures
[1/2] STEP 2/4: RUN microdnf install -y curl tar
Package                                          Repository     Size
Installing:                                                         
 tar-2:1.34-7.el9.x86_64                         baseos     896.5 kB
Upgrading:                                                          
 curl-7.76.1-31.el9.x86_64                       baseos   

Show the `make-inference` help:

In [ ]:
docker run --rm -it localhost/inference:latest hatch run prod:make-inference --help

## Test the normalized difference step in the container

Generate the normalized difference between the green and nir bands:

In [ ]:
podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/crop_green.tif,target=/inputs/crop_green.tif,readonly \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/crop_nir.tif,target=/inputs/crop_nir.tif,readonly \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/norm-diff:latest \
    python \
    -m \
    app \
    /inputs/crop_green.tif \
    /inputs/crop_nir.tif

List the outputs:

In [ ]:
tree ${RUNTIME}